In [16]:
# === Bootstrap cell: shared config, paths, JSON I/O, and GitHub REST/GraphQL helpers ===

import os, json, time, re, random, pathlib
from typing import Dict, Any, Optional, Tuple, List
from datetime import datetime, timezone

import requests
import yaml
from dotenv import load_dotenv

# -------------------------------------------------------------------
# Locate repo root reliably (works no matter where you open the .ipynb)
# -------------------------------------------------------------------
def _find_repo_root(start: pathlib.Path) -> pathlib.Path:
    cur = start.resolve()
    for _ in range(8):  # walk up to 8 levels
        if (cur / "config" / "config.yaml").exists():
            return cur
        cur = cur.parent
    raise FileNotFoundError("Couldn't locate repo root (no config/config.yaml found upward).")

REPO_ROOT = _find_repo_root(pathlib.Path.cwd())
CONFIG_PATH = REPO_ROOT / "config" / "config.yaml"

# -------------------------------------------------------------------
# Load config + token
# -------------------------------------------------------------------
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CFG: Dict[str, Any] = yaml.safe_load(f) or {}

load_dotenv(REPO_ROOT / ".env")  # local only; do not commit .env
TOKEN_ENV = (CFG.get("github_token_env") or "GITHUB_TOKEN").strip()
GITHUB_TOKEN = os.getenv(TOKEN_ENV, "").strip()
if not GITHUB_TOKEN:
    raise RuntimeError(f"Missing token in environment variable {TOKEN_ENV}. "
                       f"Create .env from .env.example and set {TOKEN_ENV}=...")

# Repo + output root
REPO = CFG["repo"]  # "owner/name"
OWNER, NAME = REPO.split("/", 1)
OUT_ROOT = pathlib.Path(CFG.get("out_root") or NAME)  # "<name>" if null

# Behavior
OVERWRITE = bool(CFG.get("overwrite", True))
VERBOSE = bool(CFG.get("verbose_logs", True))

# Networking knobs
REQ_TIMEOUT = int(CFG.get("request_timeout_sec", 30))
MAX_RETRIES = int(CFG.get("max_retries", 4))
BACKOFF_BASE_MS = int(CFG.get("backoff_base_ms", 400))
BACKOFF_JITTER_MS = int(CFG.get("backoff_jitter_ms", 250))
RESPECT_RL = bool(CFG.get("respect_rate_limits", True))

# -------------------------------------------------------------------
# Tiny logging helpers
# -------------------------------------------------------------------
def log(msg: str) -> None:
    if VERBOSE:
        print(msg, flush=True)

def warn(msg: str) -> None:
    print(f"⚠️  {msg}", flush=True)

print("Using repo root:", REPO_ROOT)

# -------------------------------------------------------------------
# Canonical output layout helpers
# -------------------------------------------------------------------
def ensure_dir(p: pathlib.Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def repo_root() -> pathlib.Path:
    root = OUT_ROOT
    ensure_dir(root)
    return root

def tags_all_json() -> pathlib.Path:
    d = repo_root() / "tags"
    ensure_dir(d)
    return d / "tags.all.json"

def series_dir(kind: str, series: str) -> pathlib.Path:
    d = repo_root() / kind / series
    ensure_dir(d)
    return d

def pair_stem(base: str, compare: str) -> str:
    return f"{base}...{compare}"

def pair_json(series: str, stem: str, kind: str) -> pathlib.Path:
    # kind ∈ {"compare","commits","pulls","issues"}
    return series_dir(kind, series) / f"{stem}.{kind}.json"

def capsule_json(series: str, stem: str) -> pathlib.Path:
    return series_dir("commits_pr_issue", series) / f"{stem}.tarce_artifacts.json"

# -------------------------------------------------------------------
# JSON I/O
# -------------------------------------------------------------------
def read_json(path: pathlib.Path) -> Optional[Dict[str, Any]]:
    if not path.exists():
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def write_json(path: pathlib.Path, payload: Dict[str, Any]) -> None:
    ensure_dir(path.parent)
    if path.exists() and not OVERWRITE:
        log(f"Skip (exists): {path}")
        return
    tmp = path.with_suffix(path.suffix + ".tmp")
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    tmp.replace(path)
    log(f"✓ Wrote {path}")

# -------------------------------------------------------------------
# GitHub HTTP session + rate-limit aware helpers
# -------------------------------------------------------------------
SESSION = requests.Session()
SESSION.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": f"notebook-pipeline/{NAME}"
})

def _parse_reset_epoch(headers: Dict[str, Any]) -> Optional[int]:
    try:
        return int(headers.get("X-RateLimit-Reset") or headers.get("x-ratelimit-reset"))
    except Exception:
        return None

def _maybe_sleep_for_reset(resp: requests.Response) -> None:
    if not RESPECT_RL:
        return
    remaining = resp.headers.get("X-RateLimit-Remaining") or resp.headers.get("x-ratelimit-remaining")
    if remaining is not None and str(remaining).isdigit() and int(remaining) <= 0:
        reset_epoch = _parse_reset_epoch(resp.headers)
        if reset_epoch:
            now = int(time.time())
            delta = max(0, reset_epoch - now) + 1
            warn(f"Rate limit reached. Sleeping ~{delta}s until reset …")
            time.sleep(delta)

def _backoff_sleep(i: int) -> None:
    base = BACKOFF_BASE_MS / 1000.0
    jitter = random.uniform(0, BACKOFF_JITTER_MS / 1000.0)
    time.sleep((2 ** i) * base + jitter)

def rest_get_json(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Generic GET with backoff + rate-limit handling.
    """
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.get(url, params=params, timeout=REQ_TIMEOUT)
            # Honor 429 Retry-After if present
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            return r.json()
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GET failed after retries: {url} :: {last_err}")

def gh_graphql(query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    GitHub GraphQL POST with backoff + rate-limit handling.
    Endpoint: https://api.github.com/graphql
    """
    url = "https://api.github.com/graphql"
    payload = {"query": query, "variables": variables or {}}
    last_err = None
    for i in range(MAX_RETRIES + 1):
        try:
            r = SESSION.post(url, json=payload, timeout=REQ_TIMEOUT)
            if r.status_code == 429:
                ra = r.headers.get("Retry-After")
                if ra and ra.isdigit():
                    wait_s = int(ra)
                    warn(f"429 received. Sleeping {wait_s}s per Retry-After …")
                    time.sleep(wait_s)
                    continue
            if r.status_code >= 500:
                last_err = f"{r.status_code} {r.text[:200]}"
                _backoff_sleep(i)
                continue
            if r.status_code >= 400:
                raise RuntimeError(f"GraphQL HTTP {r.status_code}: {r.text[:500]}")
            _maybe_sleep_for_reset(r)
            data = r.json()
            if "errors" in data:
                last_err = f"GraphQL errors: {data['errors']}"
                _backoff_sleep(i)
                continue
            return data
        except Exception as e:
            last_err = str(e)
            _backoff_sleep(i)
    raise RuntimeError(f"GraphQL failed after retries: {last_err}")

def rate_limit_snapshot() -> Dict[str, Any]:
    """
    Returns current REST rate-limit bucket (printed; not written to JSON outputs).
    """
    try:
        info = rest_get_json("https://api.github.com/rate_limit")
        core = info.get("resources", {}).get("core", {})
        remaining = core.get("remaining")
        limit = core.get("limit")
        reset = core.get("reset")
        when = datetime.fromtimestamp(reset, tz=timezone.utc).isoformat() if reset else None
        snap = {"limit": limit, "remaining": remaining, "reset_epoch": reset, "reset_iso": when}
        log(f"Rate limit: {remaining}/{limit}, resets at {when}")
        return snap
    except Exception as e:
        warn(f"Rate limit snapshot failed: {e}")
        return {}

# -------------------------------------------------------------------
# Light helpers for series + ordering
# -------------------------------------------------------------------
def semver_series(tag_name: str) -> str:
    """
    Extract 'vX.Y' series from tags like 'v4.2.3', '4.2.0-rc.1', etc.
    Falls back to 'v0.0' if not parseable.
    """
    m = re.search(r'v?(\d+)\.(\d+)', tag_name or "")
    if not m:
        return "v0.0"
    return f"v{int(m.group(1))}.{int(m.group(2))}"

def sorted_pairs_by_tag_time(pairs: List[Tuple[str, str]], tag_index: Dict[str, Dict[str, Any]]) -> List[Tuple[str, str]]:
    """
    Sort (base, compare) by tag timestamps ascending when both are tags;
    unknown timestamps sort last.
    """
    def ts(tag: str) -> float:
        rec = tag_index.get(tag)
        if rec and rec.get("tag_timestamp"):
            try:
                return datetime.fromisoformat(rec["tag_timestamp"].replace("Z", "+00:00")).timestamp()
            except Exception:
                pass
        return float("inf")
    return sorted(pairs, key=lambda bc: (ts(bc[0]), ts(bc[1])))

log("Bootstrap ready ✓")


Using repo root: /Users/rambodparsi/Desktop/OSS Repository Selection/web_scrapper/commit_pr_issue_analysis
Bootstrap ready ✓


### Config echo

In [17]:
print("Repo:", REPO)
print("Output root:", repo_root())
print("Overwrite:", OVERWRITE, "| Verbose:", VERBOSE)
rate_limit_snapshot()


Repo: mastodon/mastodon
Output root: mastodon
Overwrite: True | Verbose: True
Rate limit: 3203/5000, resets at 2025-10-26T14:16:07+00:00


{'limit': 5000,
 'remaining': 3203,
 'reset_epoch': 1761488167,
 'reset_iso': '2025-10-26T14:16:07+00:00'}

### Find all compare files

In [18]:
from pathlib import Path

print("REPO:", REPO)
print("repo_root():", repo_root())

pairs_base = Path(repo_root()) / "compare"
print("looking in:", pairs_base)

compare_files = []
if pairs_base.exists():
    compare_files = sorted(pairs_base.rglob("*.compare.json"))

# broad fallback (handles older runs / alternate out_root)
if not compare_files:
    warn("No files under <name>/pairs. Broad-searching */pairs/**/*.compare.json …")
    compare_files = sorted(Path(REPO_ROOT).rglob("pairs/**/*.compare.json"))

print("compare files found:", len(compare_files))
print("series on disk:", sorted({p.parent.name for p in compare_files}))

SERIES_WHITELIST = CFG.get("series_whitelist") or []
if SERIES_WHITELIST:
    before = len(compare_files)
    compare_files = [p for p in compare_files if p.parent.name in set(SERIES_WHITELIST)]
    after = len(compare_files)
    print(f"applied whitelist {SERIES_WHITELIST} → {after}/{before} remain")

for p in compare_files[:10]:
    print(" -", p)


REPO: mastodon/mastodon
repo_root(): mastodon
looking in: mastodon/compare
compare files found: 107
series on disk: ['v4.0', 'v4.1', 'v4.2', 'v4.3', 'v4.4']
applied whitelist ['v4.0'] → 19/107 remain
 - mastodon/compare/v4.0/v4.0.0...v4.0.1.compare.json
 - mastodon/compare/v4.0/v4.0.0rc1...v4.0.0rc2.compare.json
 - mastodon/compare/v4.0/v4.0.0rc2...v4.0.0rc3.compare.json
 - mastodon/compare/v4.0/v4.0.0rc3...v4.0.0rc4.compare.json
 - mastodon/compare/v4.0/v4.0.0rc4...v4.0.0.compare.json
 - mastodon/compare/v4.0/v4.0.1...v4.0.2.compare.json
 - mastodon/compare/v4.0/v4.0.10...v4.0.11.compare.json
 - mastodon/compare/v4.0/v4.0.11...v4.0.12.compare.json
 - mastodon/compare/v4.0/v4.0.12...v4.0.13.compare.json
 - mastodon/compare/v4.0/v4.0.13...v4.0.14.compare.json


### Helper

In [19]:
from typing import Dict, Any, List, Optional

def _load(path: Path) -> Dict[str, Any]:
    j = read_json(path)
    return j if isinstance(j, dict) else {}

def _load_commits(series: str, stem: str) -> Dict[str, Any]:
    return _load(pair_json(series, stem, "commits"))

def _load_pulls(series: str, stem: str) -> Dict[str, Any]:
    return _load(pair_json(series, stem, "pulls"))

def _load_issues(series: str, stem: str) -> Dict[str, Any]:
    return _load(pair_json(series, stem, "issues"))

def _make_commit_obj(crow: Dict[str, Any], commit_title: Optional[str]) -> Dict[str, Any]:
    return {
        "sha": crow.get("sha"),
        "html_url": crow.get("html_url"),
        "api_url": crow.get("api_url"),
        "author_login": crow.get("author_login"),
        "commit_author_date": crow.get("commit_author_date"),
        "commit_title": commit_title
    }


### Build capsules and write to commits_pr_issue/...tarce_artifacts.json

In [20]:
# M6 — Build capsules with PR→Issue priority for GraphQL 'closes'
from pathlib import Path
from typing import Dict, Any, List

written = 0
skipped = 0

for cmp_path in compare_files:
    compay = read_json(cmp_path)
    if not compay:
        continue

    series = cmp_path.parent.name
    base = compay.get("base")
    comp = compay.get("compare")
    stem = pair_stem(base, comp)
    out_path = capsule_json(series, stem)

    if out_path.exists() and not OVERWRITE:
        log(f"Skip (exists): {out_path}")
        skipped += 1
        continue

    # Load the three auxiliary files (may be missing; we keep structure even if empty)
    commits_j = read_json(pair_json(series, stem, "commits")) or {}
    pulls_j   = read_json(pair_json(series, stem, "pulls"))   or {}
    issues_j  = read_json(pair_json(series, stem, "issues"))  or {}

    # --- Indexes ---
    # 1) commit sha -> (title, pr numbers in order from commits.json)
    sha_to_title: Dict[str, str] = {}
    sha_to_prs: Dict[str, List[int]] = {}
    for it in (commits_j.get("items") or []):
        sha = it.get("sha")
        sha_to_title[sha] = it.get("commit_title")
        nums = [r.get("number") for r in (it.get("pr_refs") or []) if isinstance(r.get("number"), int)]
        sha_to_prs[sha] = nums

    # 2) pr number -> pr object; also collect PR→issue refs with relationship (closes|references)
    pr_map: Dict[int, Dict[str, Any]] = {}
    pr_to_issue_meta: Dict[int, List[Dict[str, Any]]] = {}
    for pr in (pulls_j.get("items") or []):
        n = pr.get("number")
        if isinstance(n, int):
            pr_map[n] = pr
            refs = []
            for r in (pr.get("issue_refs") or []):
                num = r.get("number")
                rel = (r.get("relationship") or "references").lower()
                url = r.get("html_url")
                if isinstance(num, int):
                    refs.append({"number": num, "relationship": rel, "html_url": url})
            pr_to_issue_meta[n] = refs

    # 3) issue number -> issue object (labels/title/body/state)
    issue_map: Dict[int, Dict[str, Any]] = {}
    for isj in (issues_j.get("items") or []):
        n = isj.get("number")
        if isinstance(n, int):
            issue_map[n] = isj

    # --- Build capsules per commit in compare order ---
    capsules = []
    compare_commits = compay.get("commits") or []

    for crow in compare_commits:
        sha = crow.get("sha")
        commit_obj = {
            "sha": sha,
            "html_url": crow.get("html_url"),
            "api_url": crow.get("api_url"),
            "author_login": crow.get("author_login"),
            "commit_author_date": crow.get("commit_author_date"),
            "commit_title": sha_to_title.get(sha)
        }

        # PRs for this commit
        pr_nums = sha_to_prs.get(sha, [])
        pr_objs: List[Dict[str, Any]] = []
        issue_objs: List[Dict[str, Any]] = []

        # We’ll prioritize issues that *close* first, but still include extra references
        # Aggregate across all PRs tied to this commit:
        issue_seen = set()
        issue_closes_flag: Dict[int, bool] = {}  # number -> any 'closes' from any PR
        issue_source_url: Dict[int, str] = {}

        for pn in pr_nums:
            pr = pr_map.get(pn)
            if pr:
                # materialize PR object (labels, title/body; issue_refs remain attached to PR)
                pr_objs.append({
                    "number": pr.get("number"),
                    "html_url": pr.get("html_url"),
                    "api_url": pr.get("api_url"),
                    "state": pr.get("state"),
                    "merged_at": pr.get("merged_at"),
                    "pr_title": pr.get("pr_title"),
                    "pr_body": pr.get("pr_body"),
                    "labels": list(pr.get("labels") or []),
                    "issue_refs": pr.get("issue_refs") or []
                })

                # capture issue refs + relationship from this PR
                for meta in pr_to_issue_meta.get(pn, []):
                    inum = meta["number"]
                    issue_source_url[inum] = meta.get("html_url") or issue_source_url.get(inum)
                    if meta.get("relationship") == "closes":
                        issue_closes_flag[inum] = True
                    issue_seen.add(inum)
            else:
                # placeholder PR if pulls.json missing this number
                pr_objs.append({
                    "number": pn,
                    "html_url": f"https://github.com/{OWNER}/{NAME}/pull/{pn}",
                    "api_url": f"https://api.github.com/repos/{OWNER}/{NAME}/pulls/{pn}",
                    "state": None,
                    "merged_at": None,
                    "pr_title": None,
                    "pr_body": None,
                    "labels": [],
                    "issue_refs": []
                })

        # Now materialize issues: sort so any with 'closes' appear first
        issue_nums_sorted = sorted(list(issue_seen), key=lambda x: (0 if issue_closes_flag.get(x) else 1, x))
        for inum in issue_nums_sorted:
            src = issue_map.get(inum)
            if src:
                issue_objs.append({
                    "number": src.get("number"),
                    "html_url": src.get("html_url") or issue_source_url.get(inum) or f"https://github.com/{OWNER}/{NAME}/issues/{inum}",
                    "state": src.get("state"),
                    "issue_title": src.get("issue_title"),
                    "issue_body": src.get("issue_body"),
                    "labels": list(src.get("labels") or [])
                })
            else:
                # placeholder if issues file missing or specific issue not present
                issue_objs.append({
                    "number": inum,
                    "html_url": issue_source_url.get(inum) or f"https://github.com/{OWNER}/{NAME}/issues/{inum}",
                    "state": None,
                    "issue_title": None,
                    "issue_body": None,
                    "labels": []
                })

        capsules.append({
            "commit": [commit_obj],  # your schema: commit as single-item list
            "prs": pr_objs,
            "issues": issue_objs
        })

    # counts (unchanged semantics)
    commits_count = len(compare_commits)
    prs_count = len({p.get("number") for p in (pulls_j.get("items") or []) if isinstance(p.get("number"), int)})
    issues_count = len({i.get("number") for i in (issues_j.get("items") or []) if isinstance(i.get("number"), int)})
    commit_pr = sum(1 for cap in capsules if len(cap.get("prs") or []) > 0)
    commit_pr_issue = sum(1 for cap in capsules if (len(cap.get("prs") or []) > 0 and len(cap.get("issues") or []) > 0))

    payload = {
        "repo": REPO,
        "base": base,
        "compare": comp,
        "counts": {
            "commits": commits_count,
            "prs": prs_count,
            "issues": issues_count,
            "commit_pr": commit_pr,
            "commit_pr_issue": commit_pr_issue
        },
        "source_files": {
            "pairs": str(Path("pairs")   / series / f"{stem}.compare.json"),
            "commits": str(Path("commits") / series / f"{stem}.commits.json"),
            "pulls": str(Path("pulls")   / series / f"{stem}.pulls.json"),
            "issues": str(Path("issues") / series / f"{stem}.issues.json")
        },
        "commit_pr_issue": capsules
    }

    write_json(out_path, payload)
    written += 1

print(f"Done. Written: {written} | Skipped: {skipped}")
rate_limit_snapshot()


✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.0...v4.0.1.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.0rc1...v4.0.0rc2.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.0rc2...v4.0.0rc3.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.0rc3...v4.0.0rc4.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.0rc4...v4.0.0.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.1...v4.0.2.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.10...v4.0.11.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.11...v4.0.12.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.12...v4.0.13.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.13...v4.0.14.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.14...v4.0.15.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.2...v4.0.3.tarce_artifacts.json
✓ Wrote mastodon/commits_pr_issue/v4.0/v4.0.3...v4.0.4.ta

{'limit': 5000,
 'remaining': 3203,
 'reset_epoch': 1761488167,
 'reset_iso': '2025-10-26T14:16:07+00:00'}